CS4001/4042 Assignment 1, Part B, Q1
---

Real world datasets often have a mix of numeric and categorical features – this dataset is one example. To build models on such data, categorical features have to be encoded or embedded.

PyTorch Tabular is a library that makes it very convenient to build neural networks for tabular data. It is built on top of PyTorch Lightning, which abstracts away boilerplate model training code and makes it easy to integrate other tools, e.g. TensorBoard for experiment tracking.

For questions B1 and B2, the following features should be used:   
- **Numeric / Continuous** features: dist_to_nearest_stn, dist_to_dhoby, degree_centrality, eigenvector_centrality, remaining_lease_years, floor_area_sqm
- **Categorical** features: month, town, flat_model_type, storey_range



---



In [2]:
# !pip install pytorch_tabular[extra]

In [3]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

> Divide the dataset (‘hdb_price_prediction.csv’) into train, validation and test sets by using entries from year 2019 and before as training data, year 2020 as validation data and year 2021 as test data.
**Do not** use data from year 2022 and year 2023.



In [9]:
# TODO: Enter your code here

df = pd.read_csv('hdb_price_prediction.csv')
df_train = df[df['year'] <= 2019]
df_val = df[df['year'] == 2020]
df_test = df[df['year'] == 2021]


> Refer to the documentation of **PyTorch Tabular** and perform the following tasks: https://pytorch-tabular.readthedocs.io/en/latest/#usage
- Use **[DataConfig](https://pytorch-tabular.readthedocs.io/en/latest/data/)** to define the target variable, as well as the names of the continuous and categorical variables.
- Use **[TrainerConfig](https://pytorch-tabular.readthedocs.io/en/latest/training/)** to automatically tune the learning rate. Set batch_size to be 1024 and set max_epoch as 50.
- Use **[CategoryEmbeddingModelConfig](https://pytorch-tabular.readthedocs.io/en/latest/models/#category-embedding-model)** to create a feedforward neural network with 1 hidden layer containing 50 neurons.
- Use **[OptimizerConfig](https://pytorch-tabular.readthedocs.io/en/latest/optimizer/)** to choose Adam optimiser. There is no need to set the learning rate (since it will be tuned automatically) nor scheduler.
- Use **[TabularModel](https://pytorch-tabular.readthedocs.io/en/latest/tabular_model/)** to initialise the model and put all the configs together.

In [18]:
# TODO: Enter your code here
num_col_names = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
cat_col_names = ['month', 'town', 'flat_model_type', 'storey_range']
data_config = DataConfig(
    target=[
        'resale_price'
    ],  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=50,
)
optimizer_config = OptimizerConfig('Adam')

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="10-50-1",  # Number of nodes in each layer
    activation="LeakyReLU",  # Activation between each layers
    learning_rate=1e-3,
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)
tabular_model.fit(train=df_train, validation=df_val)
result = tabular_model.evaluate(df_test)
pred_df = tabular_model.predict(df_test)
tabular_model.save_model("hdb_regression")
loaded_model = TabularModel.load_from_checkpoint("hdb_regression")

2023-09-18 00:08:12,199 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-09-18 00:08:12,239 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-09-18 00:08:12,244 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task


2023-09-18 00:08:12,357 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-09-18 00:08:12,534 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-09-18 00:08:12,576 - {pytorch_tabular.tabular_model:573} - INFO - Auto LR Find Started
c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:613: UserWarning: Checkpoint directory D:\micha\Michael\GitHub\Mic

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.8317637711026709
Restoring states from the checkpoint path at d:\micha\Michael\GitHub\Michael_Lee\CZ4042_lab\Assignment\.lr_find_b9129298-5a20-4eae-bbbd-912fb49e9866.ckpt
Restored all states from the checkpoint file at d:\micha\Michael\GitHub\Michael_Lee\CZ4042_lab\Assignment\.lr_find_b9129298-5a20-4eae-bbbd-912fb49e9866.ckpt
2023-09-18 00:08:17,527 - {pytorch_

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  1.2 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │      2 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

...

c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: 
You called `self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: 
You called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable 
one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       4164228096.0        │
│  test_mean_squared_error  │       4164228096.0        │
└───────────────────────────┴───────────────────────────┘

Output()

2023-09-18 00:09:05,222 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
2023-09-18 00:09:05,232 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


> Report the test RMSE error and the test R2 value that you obtained.



In [26]:
from sklearn.metrics import r2_score, mean_squared_error
rsme = pow(mean_squared_error(pred_df['resale_price'], pred_df['resale_price_prediction']), 0.5)
r2_score = r2_score(pred_df['resale_price'], pred_df['resale_price_prediction'])

print("RSME Score:", rsme)
print("R2 Score:", r2_score)


RSME Score: 64530.830684105415
R2 Score: 0.8425738921477324


RSME Score: 64530

R2 Score: 0.84


> Print out the corresponding rows in the dataframe for the top 25 test samples with the largest errors. Identify a trend in these poor predictions and suggest a way to reduce these errors.



In [31]:
# TODO: Enter your code here
pred_df['pred_error'] = abs(pred_df['resale_price_prediction'] - pred_df['resale_price'])
pred_df = pred_df.sort_values('pred_error', ascending=False)
pred_df.head(25).reset_index()


,index,month,year,town,full_address,nearest_stn,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price,resale_price_prediction,pred_error
0,106192,12,2021,QUEENSTOWN,89 DAWSON ROAD,Queenstown,0.658035,3.807573,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.333333,109.0,04 TO 06,968000.0,505846.59375,462153.40625
1,105695,6,2021,QUEENSTOWN,91 DAWSON ROAD,Queenstown,0.745596,3.720593,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.916667,97.0,07 TO 09,930000.0,467928.68750,462071.31250
2,106057,10,2021,QUEENSTOWN,92 DAWSON ROAD,Queenstown,0.584731,3.882019,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.500000,97.0,13 TO 15,958000.0,496388.62500,461611.37500
3,105468,3,2021,QUEENSTOWN,90 DAWSON ROAD,Queenstown,0.701852,3.763948,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",94.166667,95.0,10 TO 12,920000.0,475262.81250,444737.18750
4,105699,6,2021,QUEENSTOWN,91 DAWSON ROAD,Queenstown,0.745596,3.720593,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.916667,109.0,31 TO 33,1032888.0,597809.87500,435078.12500
5,105696,6,2021,QUEENSTOWN,89 DAWSON ROAD,Queenstown,0.658035,3.807573,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.916667,109.0,10 TO 12,950000.0,528396.00000,421604.00000
6,106191,12,2021,QUEENSTOWN,89 DAWSON ROAD,Queenstown,0.658035,3.807573,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.416667,97.0,19 TO 21,950000.0,528797.25000,421202.75000
7,106190,12,2021,QUEENSTOWN,92 DAWSON ROAD,Queenstown,0.584731,3.882019,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.333333,97.0,16 TO 18,938888.0,517850.96875,421037.03125
8,105961,9,2021,QUEENSTOWN,90 DAWSON ROAD,Queenstown,0.701852,3.763948,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.583333,97.0,10 TO 12,900000.0,482565.59375,417434.40625
9,105966,9,2021,QUEENSTOWN,91 DAWSON ROAD,Queenstown,0.745596,3.720593,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.666667,97.0,25 TO 27,968800.0,552984.93750,415815.06250


The top 25 test samples with the largest errors, the following trends were observed.

- For the categorical features, most of the 25 samples were from the same town ie. QUEENSTOWN. They were also located along DAWSON ROAD. Additionally, the nearest_stn was Queenstown, and the the flat_model_type of these houses were of type "4 ROOM, Premium Apartment Loft". The exception to these trends would be data at row 18, where the town this house was located in was BUKIT MERAH, with Queenstown being the nearest_stn, and flat_model_type "3 ROOM, Standard".

- For their numerical features, the degree_centrality, eigenvector_centrality, dist_to_nearest_stn, dist_to_dhoby, and floor_area_sqm are all similiar in values.	

Improvements to reduce errors.
- Normalisation of continous data can be performed. In the target label 'resale_price', the values are rather large compared to the data features the model is trained with. This could have contributed to the large errors observed. 
- Feature engineering can be done to both remove redundant features and also create features that will provide more useful information during training. 
- Different encoding techniques for categorical features can be studied. 